<a href="https://colab.research.google.com/github/TrongNgoVan/Nam-4_Ky-1_NhapMonKhoaHocDuLieu-PTIT/blob/master/GNN_LSTM_Neo4J.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>